# Classical Music classification using neural networks


In [18]:
import numpy as np
import librosa
import librosa.feature
import librosa.display
import glob
import matplotlib.pyplot as plt
from keras.models import Sequential

from pathlib import Path
from sys import argv
import subprocess
import shlex
import zipfile, urllib.request, shutil

import zipfile, urllib.request, shutil
import os

In [19]:
def display_mfcc(song):
    y,_ = librosa.load(song)
    mfcc = librosa.feature.mfcc(y)

    plt.figure(figsize=(10, 4))
    librosa.display.specshow(mfcc, x_axis='time', y_axis='mel')
    plt.colorbar()
    plt.title(song)
    plt.tight_layout()
    plt.show()


def convert_all_midi_to_wav(folder_path):
    files = [file for file in folder_path.iterdir() if file.is_file() and ("mid" in file.name)]
    for (x, file) in enumerate(files):
        without_file_extension = file.name[:-4]
        resolved = Path(f'{folder_path}/{without_file_extension}')
        # synth adds .mid to input file by default, so we remove it first
        if (Path(f'{resolved}.wav').is_file() == False):
            # Check if .wav exists, so we can safely re-run the file without re-converting
            # any files that were already converted
            print('Converting...', without_file_extension)
            subprocess.call(shlex.split(f'./convert_midi_to_wav.sh {resolved}'))


def do_recursively_on_folders(dir_path, action):
    folders = [folder for folder in dir_path.iterdir() if folder.is_dir()]
    for folder in folders:
        action(folder)

In [20]:
display_mfcc()

TypeError: display_mfcc() missing 1 required positional argument: 'song'

In [57]:
# imports classical piano music from classical piano http://www.piano-midi.de/midi_files.htm
def import_classical_piano_music():
    urls = ['http://www.piano-midi.de/zip/albeniz.zip',
            'http://www.piano-midi.de/zip/bach.zip',
            'http://www.piano-midi.de/zip/balakir.zip',
            'http://www.piano-midi.de/zip/beeth.zip',
            'http://www.piano-midi.de/zip/borodin.zip',
            'http://www.piano-midi.de/zip/brahms.zip',
            'http://www.piano-midi.de/zip/burgm.zip',
            'http://www.piano-midi.de/zip/chopin.zip',
            'http://www.piano-midi.de/zip/debussy.zip',
            'http://www.piano-midi.de/zip/granados.zip',
            'http://www.piano-midi.de/zip/grieg.zip',
            'http://www.piano-midi.de/zip/haydn.zip',
            'http://www.piano-midi.de/zip/liszt.zip',
            'http://www.piano-midi.de/zip/mendelssohn.zip',
            'http://www.piano-midi.de/zip/mozart.zip',
            'http://www.piano-midi.de/zip/muss.zip',
            'http://www.piano-midi.de/zip/schubert.zip',
            'http://www.piano-midi.de/zip/schumann.zip',
            'http://www.piano-midi.de/zip/tschai.zip']
    directory_to_unzip_to = 'PianoMidi'

    for url in urls:
        file_name = url[29:]
        with urllib.request.urlopen(url) as response, open(file_name, 'wb') as out_file:
            shutil.copyfileobj(response, out_file)
            try:
                with zipfile.ZipFile(file_name) as zf:
                    zf.extractall(path=directory_to_unzip_to)
                    print("processed: " + file_name)
            except zipfile.BadZipFile:
                print("could not process: " + file_name + " (BadZipFile)")
        os.remove(file_name)    

In [58]:
import_classical_piano_music()

processed: albeniz.zip
could not process: bach.zip (BadZipFile)
processed: balakir.zip
processed: beeth.zip
processed: borodin.zip
processed: brahms.zip
processed: burgm.zip
processed: chopin.zip
processed: debussy.zip
processed: granados.zip
processed: grieg.zip
could not process: haydn.zip (BadZipFile)
could not process: liszt.zip (BadZipFile)
could not process: mendelssohn.zip (BadZipFile)
could not process: mozart.zip (BadZipFile)
could not process: muss.zip (BadZipFile)
processed: schubert.zip
processed: schumann.zip
processed: tschai.zip
